# 0 - LIB IMPORTS

In [111]:
import pandas as pd
from plyer import notification
import datetime
import requests
import sqlite3
from sqlite3 import InterfaceError

# 1 - ALERT FUNCTION DEFINITION

In [97]:
def alert(level, database, step):

    if level not in [1,2,3]:
        print(f"Alert level [{level}] not recognized, must be 1, 2 or 3.")

    else:

        if level == 1:
            alertTitle = "Attention: Low Alert"

        elif level == 2:
            alertTitle = "Attention: Medium Alert"

        else:
            alertTitle = "ATTENTION: HIGH ALERT !!"

        alertDate = datetime.datetime.now()
        alertMessage = f"\nFailed to load database [{database}] at the step [{step}]\nTime of Alert: [{alertDate}]"

        notification.notify(title = alertTitle,
                            message = alertMessage,
                            timeout = 10)

# 2 - EXTRACTION

### 2.1 - GET FROM COUNTRIES API


In [98]:
COUNTRIES_URL = "https://restcountries.com/v3.1/all"

try:
    response = requests.get(COUNTRIES_URL)

    if response.status_code == 200:
        jsonResponse = response.json()
        countries_df = pd.DataFrame(jsonResponse)

    else:
        print("Could not get response, Status Code:", response.status_code)
        alert(3, "countries_dataframe", "API URL Get")

except requests.exceptions.RequestException as err:
    print(err)
    alert(3, "countries_dataframe", "API URL Get")

### - 2.2 - FIRST ANALYSIS

In [99]:
countries_df.columns

Index(['name', 'tld', 'cca2', 'ccn3', 'cca3', 'cioc', 'independent', 'status',
       'unMember', 'currencies', 'idd', 'capital', 'altSpellings', 'region',
       'subregion', 'languages', 'translations', 'latlng', 'landlocked',
       'area', 'demonyms', 'flag', 'maps', 'population', 'gini', 'fifa', 'car',
       'timezones', 'continents', 'flags', 'coatOfArms', 'startOfWeek',
       'capitalInfo', 'postalCode', 'borders'],
      dtype='object')

In [100]:
countries_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 35 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   name          250 non-null    object 
 1   tld           249 non-null    object 
 2   cca2          250 non-null    object 
 3   ccn3          249 non-null    object 
 4   cca3          250 non-null    object 
 5   cioc          206 non-null    object 
 6   independent   249 non-null    object 
 7   status        250 non-null    object 
 8   unMember      250 non-null    bool   
 9   currencies    247 non-null    object 
 10  idd           250 non-null    object 
 11  capital       246 non-null    object 
 12  altSpellings  250 non-null    object 
 13  region        250 non-null    object 
 14  subregion     245 non-null    object 
 15  languages     249 non-null    object 
 16  translations  250 non-null    object 
 17  latlng        250 non-null    object 
 18  landlocked    250 non-null    

# 3 - TRANSFORMATION

### 3.1 - DATAFRAME SEPARATION

In [101]:
try:

    countries_name = [resp["common"] for resp in countries_df["name"]]
    is_un_member = [resp for resp in countries_df["unMember"]]
    population = [resp for resp in countries_df["population"]]
    capitals = [resp for resp in countries_df["capital"]]

    continents = [resp for resp in countries_df["continents"]]
    areas = [resp for resp in countries_df["area"]]
    borders = [resp for resp in countries_df["borders"]]

    region = [resp for resp in countries_df["region"]]
    subregion = [resp for resp in countries_df["subregion"]]
    timezones = [resp for resp in countries_df["timezones"]]

    social_status_df = pd.DataFrame({
        "Name": countries_name,
        "Capital": capitals,
        "Is_UN_member": is_un_member,
        "Population": population
    })

    borders_df = pd.DataFrame({
        "Name": countries_name,
        "Continent": continents,
        "Area": areas,
        "Border_with": borders
    })

    region_time_df = pd.DataFrame({
        "Name": countries_name,
        "Region": region,
        "Subregion": subregion,
        "Timezones": timezones
    })

except KeyError as err:

    print("Column not found:", err)
    alert(2, "countries_dataframe", "Data sub-df creation")

In [102]:
social_status_df.head(5)

,Name,Capital,Is_UN_member,Population
0,Cyprus,[Nicosia],True,1207361
1,Eritrea,[Asmara],True,5352000
2,Liberia,[Monrovia],True,5057677
3,Bermuda,[Hamilton],False,63903
4,Vatican City,[Vatican City],True,451


In [103]:
borders_df.head(5)

,Name,Continent,Area,Border_with
0,Cyprus,[Europe],9251.00,NaN
1,Eritrea,[Africa],117600.00,"[DJI, ETH, SDN]"
2,Liberia,[Africa],111369.00,"[GIN, CIV, SLE]"
3,Bermuda,[North America],54.00,NaN
4,Vatican City,[Europe],0.44,[ITA]


In [104]:
region_time_df.head(5)

,Name,Region,Subregion,Timezones
0,Cyprus,Europe,Southern Europe,[UTC+02:00]
1,Eritrea,Africa,Eastern Africa,[UTC+03:00]
2,Liberia,Africa,Western Africa,[UTC]
3,Bermuda,Americas,North America,[UTC-04:00]
4,Vatican City,Europe,Southern Europe,[UTC+01:00]


### 3.2 - TREATING NaN

In [105]:
fillNa = {'Border_with': 'Island'}
borders_df = borders_df.fillna(fillNa)

fillNa = {'Capital': 'International'}
social_status_df = social_status_df.fillna(fillNa)

### 3.3 - REMOVING DUPLICATES

In [106]:
try:

    social_status_df['Capital'] = social_status_df['Capital'].apply(lambda x: x[0] if isinstance(x, list) else x)
    borders_df['Continent'] = borders_df['Continent'].apply(lambda x: x[0] if isinstance(x, list) else x)

    social_status_df = social_status_df.drop_duplicates(['Name', 'Capital', 'Is_UN_member', 'Population'])
    borders_df = borders_df.drop_duplicates(['Name', 'Continent', 'Area'])
    region_time_df = region_time_df.drop_duplicates(['Name', 'Region', 'Subregion'])

except TypeError as err:

    print("Some column is preventing from droping duplicates.")
    print("Check for column with the types:", err)
    alert(3, "countries_dataframe", "Duplicates Drop at Transformation step")

# 4 - LOAD

### 4.1 - CREATING DATABASE

In [107]:
def save_db(df: pd.DataFrame, table_name, db_name):
    conn = sqlite3.connect(db_name)
    df.to_sql(table_name, conn, if_exists = "replace", index = False)
    conn.close()
    return True

def read_table(db_name, query):
    conn = sqlite3.connect(db_name)
    result = pd.read_sql(query, conn)
    conn.close()
    return result

### 4.2 - LOADING DATAFRAMES TO TABLES

In [114]:
try:

    region_time_df['Timezones'] = region_time_df['Timezones'].astype(str)
    borders_df['Border_with'] = borders_df['Border_with'].astype(str)

    save_db(borders_df, "borders", "db_projeto_final1.db")
    save_db(social_status_df, "social_status", "db_projeto_final1.db")
    save_db(region_time_df, "region_time", "db_projeto_final1.db")

except InterfaceError as err:
    
    print("Some column has unsupported value for SQL.")
    print("Check for column with non primitive types.", err)
    alert(3, "countries_dataframe", "Load df to SQL step")


### 4.3 - VALIDATING DATA

In [ ]:
query = "select name, capital, population from social_status where population < 10000 order by 3"
read_table("db_projeto_final1.db", query)

,Name,Capital,Population
0,Bouvet Island,International,0
1,Heard Island and McDonald Islands,International,0
2,South Georgia,King Edward Point,30
3,Pitcairn Islands,Adamstown,56
4,United States Minor Outlying Islands,Washington DC,300
5,French Southern and Antarctic Lands,Port-aux-Français,400
6,Vatican City,Vatican City,451
7,Cocos (Keeling) Islands,West Island,544
8,Antarctica,International,1000
9,Tokelau,Fakaofo,1411


In [ ]:
query = "select name, area from borders where border_with like '%ITA%' order by area desc"
read_table("db_projeto_final1.db", query)

,Name,Area
0,France,551695.00
1,Austria,83871.00
2,Switzerland,41284.00
3,Slovenia,20273.00
4,San Marino,61.00
5,Vatican City,0.44


In [ ]:
query = """
            select b.name, b.area, r.timezones
            from borders b
            join region_time r on r.name = b.name
            where b.border_with like '%ITA%'
            order by b.area desc
        """
read_table("db_projeto_final1.db", query)

,Name,Area,Timezones
0,France,551695.00,"['UTC-10:00', 'UTC-09:30', 'UTC-09:00', 'UTC-0..."
1,Austria,83871.00,['UTC+01:00']
2,Switzerland,41284.00,['UTC+01:00']
3,Slovenia,20273.00,['UTC+01:00']
4,San Marino,61.00,['UTC+01:00']
5,Vatican City,0.44,['UTC+01:00']


In [ ]:
joined1_df = pd.merge(social_status_df, borders_df, on="Name")
joined2_df = pd.merge(joined1_df, region_time_df, on="Name")
joined2_df

,Name,Capital,Is_UN_member,Population,Continent,Area,Border_with,Region,Subregion,Timezones
0,Cyprus,Nicosia,True,1207361,Europe,9251.00,Island,Europe,Southern Europe,['UTC+02:00']
1,Eritrea,Asmara,True,5352000,Africa,117600.00,"['DJI', 'ETH', 'SDN']",Africa,Eastern Africa,['UTC+03:00']
2,Liberia,Monrovia,True,5057677,Africa,111369.00,"['GIN', 'CIV', 'SLE']",Africa,Western Africa,['UTC']
3,Bermuda,Hamilton,False,63903,North America,54.00,Island,Americas,North America,['UTC-04:00']
4,Vatican City,Vatican City,True,451,Europe,0.44,['ITA'],Europe,Southern Europe,['UTC+01:00']
...,...,...,...,...,...,...,...,...,...,...
245,South Sudan,Juba,True,11193729,Africa,619745.00,"['CAF', 'COD', 'ETH', 'KEN', 'SDN', 'UGA']",Africa,Middle Africa,['UTC+03:00']
246,Honduras,Tegucigalpa,True,9904608,North America,112492.00,"['GTM', 'SLV', 'NIC']",Americas,Central America,['UTC-06:00']
247,Saint Vincent and the Grenadines,Kingstown,True,110947,North America,389.00,Island,Americas,Caribbean,['UTC-04:00']
248,Sri Lanka,Sri Jayawardenepura Kotte,True,21919000,Asia,65610.00,['IND'],Asia,Southern Asia,['UTC+05:30']
